<a href="https://colab.research.google.com/github/pri2k/ML-Practice-Repositories/blob/main/English%20Nagamese%20Language%20Translation/Eng_Ngm_M2M_100_with_early_stopping_100rows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

belovedorange_english_nagamese_raw_data_path = kagglehub.dataset_download('belovedorange/english-nagamese-raw-data')

print('Data source import complete.')


In [ ]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
import numpy as np

In [ ]:
import pandas as pd

# Read the CSV file
data = pd.read_csv("/kaggle/input/english-nagamese-raw-data/raw_data.csv")

# Verify the data
print(data.head())


                                             English  \
0  Paul, Silvanus, and Timothy to the church of t...   
1  We always give thanks to God for all of you as...   
2  We remember before our God and Father your wor...   
3  Brothers loved by God, we know he has chosen you,   
4  because our gospel came to you not in word onl...   

                                            Nagamese  
0  Paul aru Silvanus aru Timothy pora Isor aru Pr...  
1  Amikhan hodai apnikhan nimite Isor ke dhanyaba...  
2  Amikhan pora apni khan laga biswas laga kaam, ...  
3  Isor pora morom kora bhai khan, amikhan jane T...  
4  kilemane Isor laga kotha apni khan logote khal...  


In [ ]:
data=data[:100]

In [ ]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load model and tokenizer
model_name = "facebook/m2m100_418M"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)



config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [ ]:
# # Prepare dataset
# source_texts = list(data['English'])
# target_texts = list(data['Nagamese'])

# # Tokenize the data
# inputs = tokenizer(source_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
# labels = tokenizer(target_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")["input_ids"]

# # Adjust labels for loss calculation
# labels[labels == tokenizer.pad_token_id] = -100

# # Create DataLoader
# from torch.utils.data import DataLoader, TensorDataset

# dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], labels)
# dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# # Define optimizer
# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# # Fine-tuning loop
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [ ]:
source_texts = list(data["English"])  # Replace "English" with the actual column name for source text
target_texts = list(data["Nagamese"])  # Replace "Nagamese" with the actual column name for target text



# Split the data into training and validation sets
train_source_texts, val_source_texts, train_target_texts, val_target_texts = train_test_split(
    source_texts, target_texts, test_size=0.1, random_state=42
)


print(type(train_source_texts))
print(type(train_target_texts))
print(type(val_source_texts))
print(type(val_target_texts))


<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [ ]:

# Tokenize the training and validation data
train_inputs = tokenizer(train_source_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
train_labels = tokenizer(train_target_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")["input_ids"]

val_inputs = tokenizer(val_source_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
val_labels = tokenizer(val_target_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")["input_ids"]

# Adjust labels for loss calculation
train_labels[train_labels == tokenizer.pad_token_id] = -100
val_labels[val_labels == tokenizer.pad_token_id] = -100

# Create DataLoader for training and validation
train_dataset = TensorDataset(train_inputs["input_ids"], train_inputs["attention_mask"], train_labels)
val_dataset = TensorDataset(val_inputs["input_ids"], val_inputs["attention_mask"], val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Early stopping setup
patience = 2  # number of epochs to wait for improvement before stopping
best_val_loss = float("inf")
epochs_without_improvement = 0

In [ ]:
# Fine-tuning loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validate after each epoch
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss = outputs.loss
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Epoch {epoch + 1}, Training Loss: {loss.item()}, Validation Loss: {avg_val_loss}")

    # Early stopping condition
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_without_improvement = 0
        # Save model checkpoint
        model.save_pretrained("./m2m100-finetuned-english-nagamese")
        tokenizer.save_pretrained("./m2m100-finetuned-english-nagamese")
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print("Early stopping due to no improvement in validation loss.")
        break


Epoch 1, Training Loss: 5.612968444824219, Validation Loss: 5.104864120483398


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Epoch 2, Training Loss: 4.152293682098389, Validation Loss: 4.018655776977539
Epoch 3, Training Loss: 3.7433958053588867, Validation Loss: 3.598055124282837


In [ ]:
# Final save of the fine-tuned model if early stopping was not triggered
model.save_pretrained("./m2m100-finetuned-english-nagamese-final")
tokenizer.save_pretrained("./m2m100-finetuned-english-nagamese-final")

('./m2m100-finetuned-english-nagamese-final/tokenizer_config.json',
 './m2m100-finetuned-english-nagamese-final/special_tokens_map.json',
 'm2m100-finetuned-english-nagamese-final/vocab.json',
 'm2m100-finetuned-english-nagamese-final/sentencepiece.bpe.model',
 './m2m100-finetuned-english-nagamese-final/added_tokens.json')

In [ ]:
# epochs = 3
# for epoch in range(epochs):
#     model.train()
#     i = 0
#     for batch in dataloader:
#         input_ids, attention_mask, labels = [x.to(device) for x in batch]
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#         print("iterating ", i)
#         i = i+1
#     print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# # Save the fine-tuned model
# model.save_pretrained("./m2m100-finetuned-english-nagamese")
# tokenizer.save_pretrained("./m2m100-finetuned-english-nagamese")